# Fine-tuning with Low-Rank Adaptation (LoRA)

In [ ]:
!pip install wandb python-dotenv datasets peft

In [1]:
import wandb
from dotenv import load_dotenv

from peft import LoraConfig, TaskType, get_peft_model
import transformers
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import f1_score

transformers.logging.set_verbosity_error()

In [ ]:
load_dotenv()
wandb.login()
wandb.init(project='semeval-21-task-7')

## Dataset loading

In [4]:
dataset = (
    load_dataset('csv', data_files={'train': '../data/train.csv', 'dev': '../data/dev.csv', 'test': '../data/test.csv'})
    .remove_columns(['id', 'humor_rating', 'humor_controversy', 'offense_rating', 'sentence_length'])
    .rename_column('is_humor', 'label'))

## Model setup

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


def tokenize(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=150)

In [ ]:
dataset = dataset.map(tokenize, batched=True).remove_columns(['text'])

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## LoRA Setup 

In [58]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'f1': f1
    }


## Training setup

In [66]:
for i in range(1, 6):
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=2 ** i,
        lora_alpha=4 ** i,
        lora_dropout=0.1,
        bias="none"
    )

    peft_model = get_peft_model(model, lora_config)
    trainable_params, all_param = peft_model.get_nb_trainable_parameters()
    
    print("------------------------------")
    print(f"r: {2 ** i} || trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param} ")
    
    trainer = Trainer(
        model=peft_model,
        args=TrainingArguments(
            output_dir='bert-lora-humor-detection',
            evaluation_strategy="steps",
            eval_steps=2,
            num_train_epochs=2,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            report_to="wandb",
            run_name=f"bert-lora-humor-{i}",
            adam_epsilon=1e-08,
            learning_rate=2e-5
        ),
        train_dataset=dataset['train'],
        eval_dataset=dataset['dev'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

In [26]:
# trainer.predict(dataset['test']).metrics

{'test_loss': 0.3577665686607361,
 'test_f1': 0.8537736197348548,
 'test_runtime': 37.9098,
 'test_samples_per_second': 21.103,
 'test_steps_per_second': 2.638}